# Toxicity Detection Pipeline

## Using MonoModelDetectToxicityChain


### Chain execution via `backend.detect`

In [ ]:
# # print directory of this notebook
# print(os.getcwd())
# print(os.path.abspath('..'))

/home/basti/Nextcloud/Documents/mindmaps/mind/projects/kideku/code/toxicity-detector/notebooks
/home/basti/Nextcloud/Documents/mindmaps/mind/projects/kideku/code/toxicity-detector


In [1]:
from pprint import pprint
import os, sys

import yaml

from dotenv import load_dotenv

from toxicity_detector import model_config, detect_toxicity
load_dotenv()

# change working directory to parent directory
# current_working_dir = os.getcwd()
# os.chdir(parent_dir)
# print(f"wd: {os.getcwd()}")

_APP_CONFIG_FILE = "../config/app_config.yaml"
# loading app config as dict from yaml 
with open(_APP_CONFIG_FILE, 'r') as file:
    app_config_dict = yaml.safe_load(file)

model_config_dict = model_config(app_config_dict)

input_text = "Peter is blöd."

result_dict = detect_toxicity(
    input_text, 
    None, #user_input_source, 
    "personalized_toxic_speech", # toxicity_type, 
    None, #context_info, 
    model_config_dict, 
    app_config_dict
)
pprint(result_dict['query']['analysis_result'])
pprint(result_dict['query']['contains_toxicity'])

# rechange working directory to original
# os.chdir(current_working_dir)

/home/basti/Nextcloud/Documents/mindmaps/mind/projects/kideku/code/toxicity-detector/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting new detection request (uuid: 98e46d98-212e-46d0-9906-210fa371f1de).
Chosen toxicity type: personalized_toxic_speech
Used model: Llama-3.2-3B over together.ai
Kontextinfo: None
Used api key name: hf_debatelab_inference_provider
Model kwargs: {'max_tokens': 512, 'temperature': 0.2, 'extra_headers': {'X-HF-Bill-To': 'DebateLabKIT'}}
('Basierend auf den vorläufigen Analyseergebnissen und der Begriffserläuterung '
 'von "Toxizität" kann ich folgern, dass der Text "Peter is blöd" toxischen '
 'Inhalt enthält.\n'
 '\n'
 'Die Analyse zeigt, dass der Text negative Begriffe verwendet, sich gegen '
 'eine einzelne Person richtet und eine abwertende Bezeichnung verwendet, die '
 'Peter herabwürdigt und seine Intelligenz oder Verstandsfähigkeit in Frage '
 'stellt. Diese Art von Kommentar kann als sexuelle Beleidigung, Aufforderung '
 'zur Selbstgefährdung, Verspottung und Beleidigung angesehen werden.\n'
 '\n'
 'Die Verwendung des Wortes "blöd" als Beschreibung von Peter ist eine Form '
 

In [ ]:
# # rechange working directory to original
# os.chdir(current_working_dir)
# print(f"wd: {os.getcwd()}")

wd: /home/sebastian/Nextcloud/Documents/mindmaps/mind/projects/kideku/code/toxicity_detector/notebooks


### Programmatic configuration

#### Setup LLM Model (via Toxicity Detector Backend)

In [2]:
import os, sys
from pydantic import SecretStr
from dotenv import load_dotenv

from toxicity_detector import get_openai_chat_model

load_dotenv()

api_key = SecretStr(os.environ.get("hf_debatelab_inference_provider", ""))
model = "meta-llama/Llama-3.2-3B-Instruct-Turbo"
base_url = "https://router.huggingface.co/together/v1"

chat_model = get_openai_chat_model(
    api_key=api_key,
    model=model,
    base_url=base_url
)



#### Construction/Invocation of the chain

In [3]:

from pprint import pprint
import os, sys

import yaml

# from langchain.globals import set_debug, set_verbose
# set_verbose(True)
# set_debug(True)

from toxicity_detector import MonoModelDetectToxicityChain

# toxicity_explication = "Personalisierte toxische Sprache umfasst Äußerungen, die Beleidigung, Bedrohung oder Belästigung einer Person beinhalten, gutheißen oder dazu aufrufen, *ohne Bezug auf die Zugehörigkeit* der Person zu einer Bevökerungsgruppe."
toxicity_explication = """
Gruppenbezogene toxische Sprache ist Sprache, die aufgrund von Gruppenzugehörigkeit angreift oder herabsetzt, die zu Gewalt oder Hass gegen Gruppen (oder Personen) aufruft und die auf bestimmten Merkmalen beruht wie körperlicher Erscheinung, Religion, Abstammung, nationale oder ethnische Herkunft, sexuelle Orientierung, Geschlechtsidentität oder andere. Sie kann in verschiedenen Sprachstilen vorkommen, auch in subtilen Formen oder wenn Humor verwendet wird.
"""

general_questions = (
    "Werden im Text negative Begriffe verwendet? Wen ja, welche?\n"
    "Richtet sich der Text gegen eine Person oder Gruppe? Wenn, ja gegen wen?\n"
)


indicators_yaml_str = """
call_to_violence: 
    name: "Gewaltaufruf"
    llm_description: |
        Wird gegenüber einer Bevölkerungsgruppe oder gegenüber einem Individuum als Repräsentant einer Gruppe zu Gewalt 
        aufgerufen oder solche Gewalttaten gutgeheißen, auch durch Euphemismen oder implizit?
victim_schaming: 
    name: "Opferverächtigung"
    llm_description: |
        Werden Opfer von Hasskriminalität oder das Konzept von Hassrede/Diskriminierung/Hasskriminalität gegenüber einer 
        Bevölkerungsgruppe verächtlicht gemacht, verharmlost oder gerechtfertigt? 
deadnaming:
    name: "Misgendering/Deadnaming"
    llm_description: |
        Enthält die Äußerung Misgendering oder Deadnaming einer Person?
animal_comparison:
    name: "Herabwürdigender Tiervergleich"
    llm_description: |
        Wird eine Bevölkerungsgruppe als Ganze oder ein Individuum als Repräsentant:in dieser Gruppe mit nicht-menschlichen 
        Tieren verglichen oder gleichgesetzt?
desease_conflation:
    name: "Krankheitsgleichsetzung"
    llm_description: |
        Wird eine Bevölkerungsgruppe als Ganze oder ein Individuum als Repräsentant:in dieser Gruppe mit einer Krankheit, Überträgern 
        von Krankheiten o.ä. (”Krebsgeschwür”, “Parasit”, “Ungeziefer”) gleichgesetzt? Wird mit (euphemistischen) Begriffen aus dem Wortfeld Krankheit zu Gewalt aufgerufen (”Säuberung”)?
"""

indicators_dict = yaml.safe_load(indicators_yaml_str)

# indicators_dict = {
#         "Erstes Merkmal": "Einer Bevölkerungsgruppe als Ganzer oder einem Individuum als Repräsentant:in einer Bevölkerungsgruppe werden negativ besetzte Eigenschaften zugeschrieben.",
#         "Zweites Merkmal": "Eine Bevölkerungsgruppe als Ganze oder ein Individuum als Repräsentant:in dieser Gruppe wird mit einer Krankheit, Überträgern von Krankheiten o.ä. (\"Krebsgeschwür\", \"Parasit\", \"Ungeziefer\") gleichgesetzt, bzw. wird mit (euphemistischen) Begriffen aus dem Wortfeld Krankheit oder Hygiene zu Gewalt aufgerufen (\"Säuberung\")." 
#     }
model_kwargs = {
    "max_tokens": 512,
    "temperature": 0.2,
    "extra_headers": {"X-HF-Bill-To": "DebateLabKIT"}
}

user_input = "Juden sind die Pest."
context_information = "Der Text wurde von einem Juden geäußert, der lediglich ein Beispiel für Judenhass anführen wollte."
# context_information = None
#user_input = "Scheiß Merkel wird den nächsten Schritt zur Destabilisierung und geplanten Vernichtung Deutschlands sicher bald anordnen!"

toxicitiy_detection_chain = MonoModelDetectToxicityChain.build(
    llms_dict={"chat_model": chat_model}, 
    indicators_dict=indicators_dict,
    **model_kwargs
)

output = toxicitiy_detection_chain.invoke({
    "system_prompt": "Du bist ein aufmerksamer und analytischer Mensch, der sich für die Wahrheit interessiert.",
    "toxicity_explication": toxicity_explication,
    "user_input": user_input,
    "user_input_source": "user",
    "general_questions": general_questions,
    "context_information": context_information,
    "indicators_dict": indicators_dict
})

In [4]:
print(output)

{'system_prompt': 'Du bist ein aufmerksamer und analytischer Mensch, der sich für die Wahrheit interessiert.', 'toxicity_explication': '\nGruppenbezogene toxische Sprache ist Sprache, die aufgrund von Gruppenzugehörigkeit angreift oder herabsetzt, die zu Gewalt oder Hass gegen Gruppen (oder Personen) aufruft und die auf bestimmten Merkmalen beruht wie körperlicher Erscheinung, Religion, Abstammung, nationale oder ethnische Herkunft, sexuelle Orientierung, Geschlechtsidentität oder andere. Sie kann in verschiedenen Sprachstilen vorkommen, auch in subtilen Formen oder wenn Humor verwendet wird.\n', 'user_input': 'Juden sind die Pest.', 'user_input_source': 'user', 'general_questions': 'Werden im Text negative Begriffe verwendet? Wen ja, welche?\nRichtet sich der Text gegen eine Person oder Gruppe? Wenn, ja gegen wen?\n', 'context_information': 'Der Text wurde von einem Juden geäußert, der lediglich ein Beispiel für Judenhass anführen wollte.', 'indicators_dict': {'call_to_violence': {'na